In [1]:
import pandas as pd
import numpy as np
import anndata as ad
import sgkit as sg
import cellink  as cl
from pathlib import Path

In [2]:
zarr_file_path = "/data/ouga/home/ag_gagneur/hoev/s_deeprvat/eva/theislab/hackathon24/chr22.dose.filtered.R2_0.8.vcz"
#genotype zarr file

## Load data as zarr file 

In [3]:
_gdata = sg.load_dataset(zarr_file_path)
del _gdata.attrs["vcf_header"]
_gdata

<xarray.Dataset>
Dimensions:               (variants: 143083, samples: 1034, FORMAT_GP_dim: 3,
                           ploidy: 2, contigs: 1, filters: 3, alleles: 2)
Dimensions without coordinates: variants, samples, FORMAT_GP_dim, ploidy,
                                contigs, filters, alleles
Data variables: (12/19)
    call_DS               (variants, samples) float32 dask.array<chunksize=(10000, 1000), meta=np.ndarray>
    call_GP               (variants, samples, FORMAT_GP_dim) float32 dask.array<chunksize=(10000, 1000, 3), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int8 dask.array<chunksize=(10000, 1000, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool dask.array<chunksize=(10000, 1000, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool dask.array<chunksize=(10000, 1000), meta=np.ndarray>
    contig_id             (contigs) object dask.array<chunksize=(1,), meta=np.ndarray>
    ...                    ...
    variant_contig        (variants) int8 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_filter        (variants, filters) bool dask.array<chunksize=(10000, 3), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_quality       (variants) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    source:            bio2zarr-0.1.1
    vcf_zarr_version:  0.2

In [4]:
# function for this to be merged soon 
X = _gdata.call_genotype.data.sum(-1).T
print(X.shape)
obs =  pd.DataFrame(index=_gdata.sample_id.data.compute())
var = {}
for k, v in _gdata.variables.items():
    if len(v.shape) == 1 and v.shape[0] == _gdata.variants.shape[0]:
        print(k)
        var[k.replace("variant_", "")] = v.data.compute()
var = pd.DataFrame(var)
var["chrom"] = _gdata.variant_contig.to_series().map({i: v for i, v in enumerate(_gdata.contig_id.data.compute())}).values
var["a0"] = _gdata.variant_allele[:, 0].data.compute().ravel()
var["a1"] = _gdata.variant_allele[:, 1].data.compute().ravel()
var.index = var["chrom"].astype(str) + "_" + var["position"].astype(str) + "_" + var["a0"].astype(str) + "_" + var["a1"].astype(str)
var.index.name = "variant_id"
first_cols = ['chrom', 'position', 'a0', 'a1']
var = var[first_cols + [c for c in var.columns if c not in first_cols]]
varm = {"filter": pd.DataFrame(_gdata.variant_filter.data.compute(), index=var.index, columns=_gdata.filter_id.data.compute())}
gdata = ad.AnnData(X=X, obs=obs, var=var, varm=varm)
gdata

(1034, 143083)
variant_AF
variant_ER2
variant_MAF
variant_R2
variant_contig
variant_id
variant_id_mask
variant_position
variant_quality


AnnData object with n_obs × n_vars = 1034 × 143083
    var: 'chrom', 'position', 'a0', 'a1', 'AF', 'ER2', 'MAF', 'R2', 'contig', 'id', 'id_mask', 'quality'
    varm: 'filter'

## Write variants to VCF for annotation 

In [5]:
cl.tl.write_variants_to_vcf(gdata)

[2024-10-08 16:12:38,545] INFO:cellink.tl._annotate_snps_genotype_data: number of variants to annotate: 143083
[2024-10-08 16:12:38,545] INFO:cellink.tl._annotate_snps_genotype_data: Writing variants to variants.vcf
